In [48]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [49]:
device =torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [50]:
print(device)

cuda


In [51]:
class CustomData(Dataset):
    def __init__(self, csv_file, transform=None, is_test=False):
        self.data_frame = pd.read_csv(csv_file)
        self.transform = transform
        self.is_test =is_test
    def __len__(self):
        return len(self.data_frame)
    def __getitem__(self, index):
        item =self.data_frame.iloc[index]

        if self.is_test:
            image = item.values.reshape(28,28).astype(np.uint8)
            label=None
        else:
            image = item[1:].values.reshape(28,28).astype(np.uint8)
            label = item.iloc[0]
        image = transforms.ToPILImage()(image)
        if self.transform is not None:
            image = self.transform(image)
        if self.is_test:
            return image
        else:
            return image, label

In [52]:
transform =transforms.Compose([
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))
])

In [53]:
train_data = CustomData(csv_file='/kaggle/input/digit-recognizer/train.csv', transform=transform, is_test=False)
test_data = CustomData(csv_file='/kaggle/input/digit-recognizer/test.csv', transform=transform, is_test=True)

In [54]:
print('Train Size: ' + str(len(train_data)) + ' , Test Size: ' + str(len(test_data)))

Train Size: 42000 , Test Size: 28000


In [55]:
batch_size = 64 
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True, num_workers = 2)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False, num_workers = 2)

In [56]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size =3, stride = 1, padding =1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size =2, stride =2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size =3, stride = 1, padding =1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size =3, stride = 1, padding =1)
        self.fc1 =nn.Linear(128*7*7, 128)
        self.droupout= nn.Dropout(0.5)
        self.fc2 =nn.Linear(128,20)
        self.fc3 = nn.Linear(20,10)

    def forward(self,x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.pool(x)
        x_size = x.size(1) * x.size(2) * x.size(3)
        x= x.view(-1, x_size)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

In [57]:
model= CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum=0.9)

In [59]:
epochs = 150
running_loss= 0.0
for epoch in range(epochs):
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:
            print(f"[{epoch + 1}, {i + 1}] Loss: {running_loss / 100: .3f}")
            running_loss = 0.0


[1, 100] Loss:  2.311
[1, 200] Loss:  2.296
[1, 300] Loss:  2.278
[1, 400] Loss:  2.238
[1, 500] Loss:  2.101
[1, 600] Loss:  1.448
[2, 100] Loss:  1.105
[2, 200] Loss:  0.509
[2, 300] Loss:  0.416
[2, 400] Loss:  0.373
[2, 500] Loss:  0.330
[2, 600] Loss:  0.319
[3, 100] Loss:  0.451
[3, 200] Loss:  0.245
[3, 300] Loss:  0.223
[3, 400] Loss:  0.224
[3, 500] Loss:  0.217
[3, 600] Loss:  0.193
[4, 100] Loss:  0.269
[4, 200] Loss:  0.181
[4, 300] Loss:  0.154
[4, 400] Loss:  0.162
[4, 500] Loss:  0.149
[4, 600] Loss:  0.146
[5, 100] Loss:  0.218
[5, 200] Loss:  0.131
[5, 300] Loss:  0.134
[5, 400] Loss:  0.132
[5, 500] Loss:  0.106
[5, 600] Loss:  0.124
[6, 100] Loss:  0.175
[6, 200] Loss:  0.114
[6, 300] Loss:  0.114
[6, 400] Loss:  0.096
[6, 500] Loss:  0.097
[6, 600] Loss:  0.096
[7, 100] Loss:  0.158
[7, 200] Loss:  0.098
[7, 300] Loss:  0.087
[7, 400] Loss:  0.092
[7, 500] Loss:  0.097
[7, 600] Loss:  0.088
[8, 100] Loss:  0.137
[8, 200] Loss:  0.091
[8, 300] Loss:  0.090
[8, 400] L

In [62]:
model.eval()
predictions = []
with torch.no_grad():
    for data in test_loader:
        data = data.to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.cpu().tolist())

submission = pd.DataFrame({
    "ImageId": range(1, len(predictions))
})

submission.to_csv('MNIST.csv', index=False)